Hi, this notebook will show you almost all the columns operation availables in Optimus. For row operation, IO, ML and DL please go to the examples folder in the repo

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys

In [5]:
sys.path.append("..")

## Columns Operations

In this notebook you can see a detailed overview ok all the columns operations available in Optimus. You can access the operation via df.cols.

In [6]:
from optimus import Optimus

In [7]:
# Create optimus
op = Optimus("pandas")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LuisA\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Create dataframe

In [8]:
df = op.create.dataframe({
        ("words", "str", True): ['  I like     fish  ', '    zombies', 'simpsons   cat lady', None],
        ("num", "int", True): [1, 2, 2, 3],
        ("animals", "str", True): ['dog', 'cat', 'frog', 'eagle'],
        ("thing", "str", True): ['housé', 'tv', 'table', 'glass'],
        ("two strings", "str", True): ['cat-car', 'dog-tv', 'eagle-tv-plus', 'lion-pc'],
        ("filter", "str", True): ['a', 'b', '1', 'c'],
        ("num 2", "string", True): ['1', '2', '3', '4'],
        ("col_array", None, True): [['baby', 'sorry'], ['baby 1', 'sorry 1'], ['baby 2', 'sorry 2'], ['baby 3', 'sorry 3']],                
        ("col_int", None, True): [[1, 2, 3], [3, 4], [5, 6, 7], [7, 8]]
})

df.print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]



## Create Columns

### Create a column with a constant value

In [9]:
df = df.cols.set("new_col_1", 1)
df.print()

ValueError: 'missing_columns' must be 'words', 'num', 'animals', 'thing', 'two strings', 'filter', 'num 2', 'col_array', 'col_int', received 'new_col_1'. 

In [ ]:
df.print()

### Append 2 or multiples dataframes columns like

In [ ]:
df_col = op.create.dataframe({("NEW", "str", True): ["q", "w", "e", "r"]})
df = df.cols.append(df_col)
df.print()

### Create multiple columns with a constant value

In [12]:
df.cols.append({"col 2": 2.22, "col 3": 3}).print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW               col 2      col 3
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)      (float64)    (int64)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------  -----------  ---------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q                  2.22          3
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w                  2.22          3
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e                  2.22          3
          

### Create multiple columns with a constant string, a new column with existing columns value and an array

In [13]:
df.cols.append({
    "just test": "test",
    "num 2": df['num'] * 2,
    "more numbers": [1, 2, 3, 4]
}).cols.select(["num", "just test", "num 2", "more numbers"]).print()

      num  just test          num 2    more numbers
  (int32)  (object)       (float64)         (int64)
---------  -----------  -----------  --------------
        1  test                   2               1
        2  test                   4               2
        2  test                   4               3
        3  test                   6               4



## Select columns

In [14]:
df.print()
columns = ["words", 1, "animals", 3]
df.cols.select(columns).print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r

words                      num  animals     thing
(object

### Select columns with a Regex

In [15]:
df.cols.select("n.*", regex=True).print()

      num       num 2
  (int32)    (string)
---------  ----------
        1           1
        2           2
        2           3
        3           4



### Select all the columns of type string

In [16]:
df.cols.select(data_type="int").print()

      num
  (int32)
---------
        1
        2
        2
        3



## Rename Column

In [17]:
df.cols.rename('num', 'number').print()

words                   number  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Rename multiple columns and uppercase all the columns

In [18]:
df.cols.rename([('num', 'number'), ("animals", "gods")], str.upper).print()

words                   NUMBER  GODS        thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Convert to lower case

In [19]:
df.cols.rename(str.lower).print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     new
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Convert to uppercase

In [20]:
df.cols.rename(str.upper).print()

WORDS                      NUM  ANIMALS     THING       TWO STRINGS    FILTER           NUM 2  COL_ARRAY              COL_INT     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



## Cast a columns

This is a opinionated way to handle column casting. 
One of the first thing that every data cleaning process need to acomplish is define a data dictionary.
Because of that we prefer to create a tuple like this:

df.cols().cast(
[("words","str"),
("num","int"),
("animals","float"),
("thing","str")]
)


In [21]:
df.cols.cast(columns=[("num", "str"), ("num 2", "int")]).cols.select(["num", "num 2"]).cols.dtypes()

{'num': 'object', 'num 2': 'int64'}

### Cast a column to string

In [22]:
df.cols.cast("num", "str").cols.dtypes()

{'words': 'object',
 'num': 'object',
 'animals': 'object',
 'thing': 'object',
 'two strings': 'object',
 'filter': 'object',
 'num 2': 'string',
 'col_array': 'object',
 'col_int': 'object',
 'NEW': 'object'}

### Cast all columns to string

In [23]:
df.cols.cast("*", "str").cols.dtypes()

{'words': 'object',
 'num': 'object',
 'animals': 'object',
 'thing': 'object',
 'two strings': 'object',
 'filter': 'object',
 'num 2': 'object',
 'col_array': 'object',
 'col_int': 'object',
 'NEW': 'object'}

## Move columns

In [24]:
df.cols.move("thing", "after", "words").print()

words                thing             num  animals     two strings    filter           num 2  col_array              col_int     NEW
(object)             (object)      (int32)  (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ----------  ---------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish      housé               1  dog         cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies              tv                  2  cat         dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady  table               2  frog        eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                     glass               3  eagle       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



## Sorting Columns

### Sort in Alphabetical order

In [25]:
df.cols.sort().print()

animals     col_array              col_int     filter      NEW               num       num 2  thing       two strings    words
(object)    (object)               (object)    (object)    (object)      (int32)    (string)  (object)    (object)       (object)
----------  ---------------------  ----------  ----------  ----------  ---------  ----------  ----------  -------------  -------------------
dog         ['baby', 'sorry']      [1, 2, 3]   a           q                   1           1  housé       cat-car        I like     fish
cat         ['baby 1', 'sorry 1']  [3, 4]      b           w                   2           2  tv          dog-tv         zombies
frog        ['baby 2', 'sorry 2']  [5, 6, 7]   1           e                   2           3  table       eagle-tv-plus  simpsons   cat lady
eagle       ['baby 3', 'sorry 3']  [7, 8]      c           r                   3           4  glass       lion-pc



### Sort in Reverse Alphabetical order

In [26]:
df.cols.sort(order="desc").print()

words                two strings    thing            num 2        num  NEW         filter      col_int     col_array              animals
(object)             (object)       (object)      (string)    (int32)  (object)    (object)    (object)    (object)               (object)
-------------------  -------------  ----------  ----------  ---------  ----------  ----------  ----------  ---------------------  ----------
I like     fish      cat-car        housé                1          1  q           a           [1, 2, 3]   ['baby', 'sorry']      dog
zombies              dog-tv         tv                   2          2  w           b           [3, 4]      ['baby 1', 'sorry 1']  cat
simpsons   cat lady  eagle-tv-plus  table                3          2  e           1           [5, 6, 7]   ['baby 2', 'sorry 2']  frog
                     lion-pc        glass                4          3  r           c           [7, 8]      ['baby 3', 'sorry 3']  eagle



## Drop columns

###  Drop one columns

In [27]:
df.cols.drop("num").print()

words                animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)             (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish      dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies              cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                     eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Drop multiple columns

In [28]:
df.cols.drop(["num", "words"]).print()

animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



## Chaining

cols and rows accessors are used to organize and encapsulate `optimus` methods, it can be helpfull when you look at the code because every line is self explained.

The past transformations were done step by step, but this can be achieved by chaining all operations into one line of code, like the cell below. This way is much more efficient and scalable because it uses all optimization issues from the lazy evaluation approach.

In [29]:
df.print()
df \
    .cols.rename([('num', 'number')]) \
    .cols.drop(["number", "words"]) \
    .cols.append({"col_10": 10}) \
    .cols.sort(order="desc") \
    .cols.upper("two strings") \
    .print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r

two strings    thing            num 2  NEW         filter

## Unnest  Columns

With unnest you can convert one column into multiple ones. it can hadle string, array and vectors

### Spark
Can split strings with split()

### Pandas
via str.split()

In [30]:
df.cols.select("two strings").print()
df.cols.select("two strings").cols.unnest("two strings", "-").print()

two strings
(object)
-------------
cat-car
dog-tv
eagle-tv-plus
lion-pc

two strings    two strings_0    two strings_1
(object)       (object)         (object)
-------------  ---------------  ---------------
cat-car        cat              car
dog-tv         dog              tv
eagle-tv-plus  eagle            tv-plus
lion-pc        lion             pc



### Only get the first element

In [31]:
df.cols.select("two strings").cols.unnest("two strings", "-", index=1).print()

two strings    two strings_1
(object)       (object)
-------------  ---------------
cat-car        car
dog-tv         tv
eagle-tv-plus  tv-plus
lion-pc        pc



### Unnest array of string

In [32]:
df.cols.select("col_array").cols.unnest("col_array").print()

col_array              col_array_0    col_array_1
(object)               (object)       (object)
---------------------  -------------  --------------
['baby', 'sorry']      ['baby',       'sorry']
['baby 1', 'sorry 1']  ['baby         1', 'sorry 1']
['baby 2', 'sorry 2']  ['baby         2', 'sorry 2']
['baby 3', 'sorry 3']  ['baby         3', 'sorry 3']



### Unnest and array of ints

In [33]:
df.cols.select("col_int").cols.unnest("col_int").print()

col_int     col_int_0    col_int_1
(object)    (object)     (object)
----------  -----------  -----------
[1, 2, 3]   [1,          2, 3]
[3, 4]      [3,          4]
[5, 6, 7]   [5,          6, 7]
[7, 8]      [7,          8]



### Spits in 3 parts

In [34]:
df.cols.select("two strings").cols.unnest(["two strings"], splits=3, separator="-").print()

two strings    two strings_0    two strings_1    two strings_2
(object)       (object)         (object)         (object)
-------------  ---------------  ---------------  ---------------
cat-car        cat              car
dog-tv         dog              tv
eagle-tv-plus  eagle            tv               plus
lion-pc        lion             pc



## Impute

### Fill missing data

In [35]:
df_fill = op.create.dataframe(A=[1.0, 2.0, op.nan, 4.0, 5.0], B=[op.nan, op.nan, 3.0, 4.0, 5.0], C=[1, op.nan, None, 2, 2])

In [36]:
df_fill.print()

          A            B            C
  (float64)    (float64)    (float64)
-----------  -----------  -----------
          1          nan            1
          2          nan          nan
        nan            3          nan
          4            4            2
          5            5            2



In [37]:
df_fill.cols.impute(["A", "B"], "continuous", "median").print()

          A            B            C
  (float64)    (float64)    (float64)
-----------  -----------  -----------
          1            4            1
          2            4          nan
          3            3          nan
          4            4            2
          5            5            2



In [38]:
df_fill.cols.impute(["C"], strategy="most_frequent").print()

          A            B            C
  (float64)    (float64)    (float64)
-----------  -----------  -----------
          1          nan            1
          2          nan            2
        nan            3            2
          4            4            2
          5            5            2



## Set values using user defined functions

In [39]:
df.print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Set a value only to numeric values in `filter`

Sometimes there are columns with numeric and string values together.

In order to solve this problem, set.numeric() function can be used to operate over just one of those types. 

In the next example we replace ever number with a string "new string"

In [40]:
def func(val, arg):
    return arg

df.set.numeric("filter", func, args="new string").print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  new string           3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



Or you could pass the value directly

In [41]:
df.set.numeric("filter", "new string").print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  new string           3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Sum a numeric value (20 in this case) to two columns

In [42]:
def func(val):
    return val + 20

df.cols.set(["num", "num 2"], func).print()

words                      num  animals     thing       two strings    filter          num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (int32)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ---------  ---------------------  ----------  ----------
I like     fish             21  dog         housé       cat-car        a                  21  ['baby', 'sorry']      [1, 2, 3]   q
zombies                     22  cat         tv          dog-tv         b                  22  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady         22  frog        table       eagle-tv-plus  1                  22  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                            23  eagle       glass       lion-pc        c                  23  ['baby 3', 'sorry 3']  [7, 8]      r



### Select row where column "filter" is "int"

In [43]:
df.rows.int("filter").print()

words                      num  animals     thing       two strings        filter       num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)         (object)    (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
simpsons   cat lady          2  frog        table       eagle-tv-plus           1           3  ['baby 2', 'sorry 2']  [5, 6, 7]   e



### Create an abstract dataframe to filter a rows where the value of column "num"> 1

In [44]:
df.rows.greater_than("num", 1).print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Create an abstract dataframe (Pandas UDF) to pass two arguments to a function a apply a sum operation

In [56]:
def func(val, arg1, arg2):
    return val + arg1 + arg2

df.cols.set("num_sum", func, args=[5, 6], default="num").print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW           num_sum
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)      (int32)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------  ---------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q                  12
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w                  13
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e                  13
                             3  eagle       glass       lion-pc        c                

### Apply a column expression to when the value of "num" or "num 2" is grater than 2

In [62]:
where = (df["num"] > 2) | (df["num 2"]> 2)
print(type(where))
df.cols.set(["num", "num 2"], 10, where=where).print()

<class 'optimus.engines.pandas.dataframe.PandasDataFrame'>
words                      num  animals     thing       two strings    filter          num 2  col_array              col_int     NEW
(object)               (int64)  (object)    (object)    (object)       (object)      (int64)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ---------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                   1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                   2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady         10  frog        table       eagle-tv-plus  1                  10  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                            10  eagle       glass       lion-pc        c                  10  ['baby 3', 'sorry 3']  [7, 8]      r



## Count Nulls

In [65]:
df_null = op.create.dataframe(session=[1,1,1,1,1,1,1], timestamp=[1,2,3,4,5,6,6], id=[None, 5.0, op.nan, None, 10.0, op.nan, op.nan])
df_null.print()

  session    timestamp           id
  (int64)      (int64)    (float64)
---------  -----------  -----------
        1            1          nan
        1            2            5
        1            3          nan
        1            4          nan
        1            5           10
        1            6          nan
        1            6          nan



In [66]:
df_null.cols.count_nulls("id")

5

In [67]:
df_null.cols.count_nulls("*")

{'session': 0, 'timestamp': 0, 'id': 5}

## Count uniques
### Spark

### Pandas


In [68]:
df.cols.count_uniques("*")

{'words': 4,
 'num': 3,
 'animals': 4,
 'thing': 4,
 'two strings': 4,
 'filter': 4,
 'num 2': 4,
 'col_array': 4,
 'col_int': 4,
 'NEW': 4}

## Unique

In [70]:
df.print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



In [71]:
df_distinct = op.create.dataframe({
    ("words", "str"): ["  I like    fish  ", "    zombies", "simpsons   cat lady", None, None],
    ("num", "int"): [1,2,2,3,0],
})
df_distinct

words 1 (object) not nullable,num 2 (int32) not nullable
⋅⋅I⋅like⋅⋅⋅⋅fish⋅⋅,1
⋅⋅⋅⋅zombies,2
simpsons⋅⋅⋅cat⋅lady,2
None,3
None,0


In [72]:
df_distinct.rows.unique("num").print()

words                 num
(object)          (int32)
--------------  ---------
I like    fish          1
zombies                 2
                        3
                        0



## Count Zeros

In [77]:
df_zeros = df_distinct
df_zeros.cols.count_zeros("*")

{'words': 0, 'num': 1}

## Column Data Types

In [78]:
df.cols.dtypes('*')

{'words': 'object',
 'num': 'int32',
 'animals': 'object',
 'thing': 'object',
 'two strings': 'object',
 'filter': 'object',
 'num 2': 'string',
 'col_array': 'object',
 'col_int': 'object',
 'NEW': 'object'}

## Replace

In [79]:
df.print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Replace "dog","cat" in column "animals" by the word "animals"

In [80]:
df.cols.replace("animals", ["dog", "cat"], "animals").print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  animals     housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  animals     tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Replace "dog-tv", "cat", "eagle", "fish" in columns "two strings","animals" by the string "animals"

In [82]:
df.cols.replace(["two strings", "animals"], ["dog-tv", "cat", "eagle", "fish"], "animals").print()

words                      num  animals     thing       two strings      filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)         (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  ---------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       animals-car      a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  animals     tv          animals          b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       animals-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  animals     glass       lion-pc          c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Replace "dog" by  "dog_1" and "cat" by "cat_1" in columns "animals"

In [85]:
df.cols.replace("animals", [("dog", "dog_1"), ("cat", "cat_1")])

words 1 (object) not nullable,num 2 (int32) not nullable,animals 3 (object) not nullable,thing 4 (object) not nullable,two strings 5 (object) not nullable,filter 6 (object) not nullable,num 2 7 (string) not nullable,col_array 8 (object) not nullable,col_int 9 (object) not nullable,NEW 10 (object) not nullable
⋅⋅I⋅like⋅⋅⋅⋅⋅fish⋅⋅,1,dog_1,housé,cat-car,a,1,"['baby',⋅'sorry']","[1,⋅2,⋅3]",q
⋅⋅⋅⋅zombies,2,cat_1,tv,dog-tv,b,2,"['baby⋅1',⋅'sorry⋅1']","[3,⋅4]",w
simpsons⋅⋅⋅cat⋅lady,2,frog,table,eagle-tv-plus,1,3,"['baby⋅2',⋅'sorry⋅2']","[5,⋅6,⋅7]",e
None,3,eagle,glass,lion-pc,c,4,"['baby⋅3',⋅'sorry⋅3']","[7,⋅8]",r


### Replace in column "animals", "dog" by "pet" 

In [86]:
df.cols.replace("animals", "dog", "pet").print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  pet         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Replace a,b,c by % in all columns

In [88]:
df.cols.replace("*", ["a", "b", "c"], "%").print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       %%t-%%r        %                    1  ['baby', 'sorry']      [1, 2, 3]   q
zom%ies                      2  %%t         tv          dog-tv         %                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   %%t l%dy          2  frog        t%%le       e%gle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  e%gle       gl%ss       lion-p%        %                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Replace 3 and 2 by 10 in a numeric columns

In [113]:
df.cols.replace('num', ["3", 2], 10, search_by="values").print()

['3', 2]
[10]
words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int64)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                     10  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady         10  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Replace 3 by 6 and 2 by 12 in a numeric columns

In [114]:
df.cols.replace('num', [("3", 6), (2, 12)], search_by="values").print()

['3', 2]
[6, 12]
words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int64)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                     12  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady         12  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Replace as words

In [116]:
df.cols.replace("animals", "dog", "animal", search_by="words").print()

words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  animal      housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



In [117]:
df.cols.replace("animals", "dog", "animal", search_by="words", output_cols="friends").print()

words                      num  animals     friends     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         animal      housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       eagle       glass       lion-pc        c                    4  ['baby 3', 

In [123]:
df.cols.replace("*", "1", "11", search_by="chars").print()

words                       num  animals     thing       two strings    filter           num 2  col_array                col_int     NEW
(object)               (object)  (object)    (object)    (object)       (object)      (object)  (object)                 (object)    (object)
-------------------  ----------  ----------  ----------  -------------  ----------  ----------  -----------------------  ----------  ----------
I like     fish              11  dog         housé       cat-car        a                   11  ['baby', 'sorry']        [11, 2, 3]  q
zombies                       2  cat         tv          dog-tv         b                    2  ['baby 11', 'sorry 11']  [3, 4]      w
simpsons   cat lady           2  frog        table       eagle-tv-plus  11                   3  ['baby 2', 'sorry 2']    [5, 6, 7]   e
None                          3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']    [7, 8]      r



In [130]:
df.cols.replace("animals", [("dog", "dog_1"), ("cat", "cat_1")], search_by="words").print()

['\\bdog\\b', '\\bcat\\b']
words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog_1       housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat_1       tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



In [132]:
df.cols.replace("animals", ["dog", "cat"], "animals", "words").print()

['\\bdog\\b', '\\bcat\\b']
words                      num  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  animals     housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  animals     tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



### Use Regex

In [144]:
df.cols.replace_regex('*', '.*[Cc]at.*', 'contains cat').print()

words                  num  animals       thing       two strings    filter           num 2  col_array              col_int     NEW
(object)           (int32)  (object)      (object)    (object)       (object)      (string)  (object)               (object)    (object)
---------------  ---------  ------------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish          1  dog           housé       contains cat   a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                  2  contains cat  tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
contains cat             2  frog          table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                         3  eagle         glass       lion-pc        c                    4  ['baby 3', 'sorry 3']  [7, 8]      r



## Nest

### Merge two columns in a column vector
#### Match the string as a word not as a substring

In [145]:
df.cols.copy("num", "num_1").cols.nest(["num", "num_1"], output_col="col_nested").print()

words                      num      num_1    col_nested  animals     thing       two strings    filter           num 2  col_array              col_int     NEW
(object)               (int32)    (int32)      (object)  (object)    (object)    (object)       (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ---------  ------------  ----------  ----------  -------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1          1            11  dog         housé       cat-car        a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2          2            22  cat         tv          dog-tv         b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2          2            22  frog        table       eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3          

### Merge two columns in a string columns

In [148]:
df.cols.nest(["animals", "two strings"], output_col="col_nested", shape="string", separator="---").print()

words                      num  animals     thing       two strings    col_nested            filter           num 2  col_array              col_int     NEW
(object)               (int32)  (object)    (object)    (object)       (object)              (object)      (string)  (object)               (object)    (object)
-------------------  ---------  ----------  ----------  -------------  --------------------  ----------  ----------  ---------------------  ----------  ----------
I like     fish              1  dog         housé       cat-car        dog---cat-car         a                    1  ['baby', 'sorry']      [1, 2, 3]   q
zombies                      2  cat         tv          dog-tv         cat---dog-tv          b                    2  ['baby 1', 'sorry 1']  [3, 4]      w
simpsons   cat lady          2  frog        table       eagle-tv-plus  frog---eagle-tv-plus  1                    3  ['baby 2', 'sorry 2']  [5, 6, 7]   e
                             3  eagle       glass       li

### Merge three columns in an array

In [151]:
df.cols.nest(["animals", "two strings", "num 2"], output_col="col_nested", shape="array").print()

NotImplementedError: Not implemented yet

## Histograms

In [82]:
df = op.load.csv("https://raw.githubusercontent.com/ironmussa/Optimus/master/examples/data/foo.csv")

In [83]:
df.table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [84]:
df.cols.hist("price", 10)

[{'count': 1, 'lower': 1.0, 'upper': 1.9},
 {'count': 1, 'lower': 1.9, 'upper': 2.8},
 {'count': 4, 'lower': 2.8, 'upper': 3.6999999999999997},
 {'count': 1, 'lower': 3.6999999999999997, 'upper': 4.6},
 {'count': 2, 'lower': 4.6, 'upper': 5.5},
 {'count': 0, 'lower': 5.5, 'upper': 6.4},
 {'count': 0, 'lower': 6.4, 'upper': 7.300000000000001},
 {'count': 6, 'lower': 7.300000000000001, 'upper': 8.200000000000001},
 {'count': 2, 'lower': 8.200000000000001, 'upper': 9.100000000000001},
 {'count': 2, 'lower': 9.100000000000001, 'upper': 10.0}]

In [85]:
df.cols.frequency("billingId")

{'billingId': [{'value': 992, 'count': 1},
  {'value': 912, 'count': 1},
  {'value': 886, 'count': 1},
  {'value': 875, 'count': 1},
  {'value': 812, 'count': 1},
  {'value': 735, 'count': 1},
  {'value': 672, 'count': 1},
  {'value': 634, 'count': 1},
  {'value': 624, 'count': 1},
  {'value': 553, 'count': 1}]}

## Statistics

### Quantile Statistics

In [86]:
print(df.cols.min("billingId"))
print(df.cols.percentile(['billingId', 'price'], [0.05, 0.25, 0.5, 0.75, 0.95]))
print(df.cols.max("billingId"))
print(df.cols.median(["billingId", "price"]))
print(df.cols.range(["billingId", "price"]))
print(df.cols.std(["billingId", "price"]))

111
{'billingId': {'0.05': 111.0, '0.25': 111.0, '0.5': 111.0, '0.75': 111.0, '0.95': 111.0}, 'price': {'0.05': 1.0, '0.25': 1.0, '0.5': 1.0, '0.75': 1.0, '0.95': 1.0}}
992
{'billingId': 111.0, 'price': 1.0}
{'billingId': {'min': 111, 'max': 992}, 'price': {'min': 1, 'max': 10}}
{'billingId': 280.19735, 'price': 2.95285}


In [87]:
print(df.cols.min("*"))

{'id': 1, 'firstName': '(((   Heinrich )))))', 'lastName': '             GALiLEI', 'billingId': 111, 'product': '110790', 'price': 1, 'birth': '1899/01/01', 'dummyCol': '#'}


In [88]:
print(df.cols.kurt("billingId"))
print(df.cols.mean("billingId"))
print(df.cols.skewness("billingId"))
print(df.cols.sum("billingId"))
print(df.cols.variance("billingId"))
print(df.cols.mad("billingId"))

-1.04115
556.0
-0.2137
10564
78510.55556
0.0


### Calculate Median Absolute deviation

In [89]:
df.cols.mad("price")

0.0

In [90]:
df.cols.mad("price", more=True)

{'mad': 0.0, 'median': 1.0}

### Calculate precentiles

In [91]:
print(df.cols.percentile(['price'], [0.05, 0.25, 0.5, 0.75, 0.95]))

{'0.05': 1.0, '0.25': 1.0, '0.5': 1.0, '0.75': 1.0, '0.95': 1.0}


### Calculate Mode

In [92]:
print(df.cols.mode(["price", "billingId"]))

[{'price': 8}, {'billingId': None}]


## String Operations

In [93]:
df.table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [94]:
df \
    .cols.trim("lastName") \
    .cols.lower("lastName") \
    .cols.upper(["product", "firstName"]) \
    .cols.reverse("firstName") \
    .table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,SIUL,alvarez$$%!,123,CAKE,10,1980/07/07,never
2,ÉRDNA,ampère,423,PIZA,8,1950/07/08,gonna
3,SLEIN,böhr//((%%,551,PIZZA,8,1990/07/09,give
4,LUAP,dirac$,521,PIZZA,8,1954/07/10,you
5,TREBLA,einstein,634,PIZZA,8,1990/07/11,up
6,OELILAG,galilei,672,AREPA,5,1930/08/12,never
7,LRAC,ga%%%uss,323,TACO,3,1970/07/13,gonna
8,DIVAD,h$$$ilbert,624,TAAACCOO,3,1950/07/14,let
9,SENNAHOJ,kepler,735,TACO,3,1920/04/22,you
10,SEMAJ,m$$ax%%well,875,TACO,3,1923/03/12,down


### Calculate the interquartile range

In [95]:
df.cols.iqr("price")

5.0

In [96]:
df.cols.iqr("price", more=True)

{'iqr': 5.0, 'q1': 3.0, 'q2': 8.0, 'q3': 8.0}

### Calculate  Zscore

In [97]:
df.cols.z_score("price").table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (double) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez$$%!,123,Cake,1.336800040638705,1980/07/07,never
2,André,Ampère,423,piza,0.6594882909731277,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,0.6594882909731277,1990/07/09,give
4,PAUL,dirac$,521,pizza,0.6594882909731277,1954/07/10,you
5,Albert,Einstein,634,pizza,0.6594882909731277,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,0.3564793335252382,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,1.0337910831908155,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,1.0337910831908155,1950/07/14,let
9,Johannes,KEPLER,735,taco,1.0337910831908155,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,1.0337910831908155,1923/03/12,down


## Cleaning and Date Operations Operations

In [98]:
df.cols.years_between("birth", "yyyyMMdd", ).table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (float) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez$$%!,123,Cake,10,None,never
2,André,Ampère,423,piza,8,None,gonna
3,NiELS,Böhr//((%%,551,pizza,8,None,give
4,PAUL,dirac$,521,pizza,8,None,you
5,Albert,Einstein,634,pizza,8,None,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,None,never
7,CaRL,Ga%%%uss,323,taco,3,None,gonna
8,David,H$$$ilbert,624,taaaccoo,3,None,let
9,Johannes,KEPLER,735,taco,3,None,you
10,JaMES,M$$ax%%well,875,taco,3,None,down


In [99]:
df.cols.remove("*", ["&", "%"]).table()

id 1 (string) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (string) nullable,product 5 (string) nullable,price 6 (string) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez$$!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Gauss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$axwell,875,taco,3,1923/03/12,down


In [100]:
df.cols.remove_accents("lastName").table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampere,423,piza,8,1950/07/08,gonna
3,NiELS,Bohr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [101]:
df.cols.remove_special_chars("lastName").table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr,551,pizza,8,1990/07/09,give
4,PAUL,dirac,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Gauss,323,taco,3,1970/07/13,gonna
8,David,Hilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,Maxwell,875,taco,3,1923/03/12,down


In [102]:
df.cols.clip("billingId", 100, 200).table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,200,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,200,pizza,8,1990/07/09,give
4,PAUL,dirac$,200,pizza,8,1954/07/10,you
5,Albert,Einstein,200,pizza,8,1990/07/11,up
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,200,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,200,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,200,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,200,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,200,taco,3,1923/03/12,down


In [103]:
df_abs = op.create.df(
    [
        ("words", "str", True),
        ("num", "int", True),
        ("animals", "str", True),
        ("thing", StringType(), True),
        ("two strings", StringType(), True),
        ("filter", StringType(), True),
        ("num 2", "string", True),
        ("col_array", ArrayType(StringType()), True),
        ("col_int", ArrayType(IntegerType()), True)

    ]
    ,
    [
        ("  I like     fish  ", -1, "dog", "housé", "cat-car", "a", "-1", ["baby", "sorry"], [1, 2, 3]),
        ("    zombies", -2, "cat", "tv", "dog-tv", "b", "-2", ["baby 1", "sorry 1"], [3, 4]),
        ("simpsons   cat lady", -2, "frog", "table", "eagle-tv-plus", "1", "3", ["baby 2", "sorry 2"], [5, 6, 7]),
        (None, 3, "eagle", "glass", "lion-pc", "c", "4", ["baby 3", "sorry 3"], [7, 8])
    ])

In [104]:
df_abs.cols.abs(["num", "num 2"]).table()

words 1 (string) nullable,num 2 (int) nullable,animals 3 (string) nullable,thing 4 (string) nullable,two strings 5 (string) nullable,filter 6 (string) nullable,num 2 7 (string) nullable,col_array 8 (array<string>) nullable,col_int 9 (array<int>) nullable
⋅⋅I⋅like⋅⋅⋅⋅⋅fish⋅⋅,1,dog,housé,cat-car,a,-1,"['baby',⋅'sorry']","[1,⋅2,⋅3]"
⋅⋅⋅⋅zombies,2,cat,tv,dog-tv,b,-2,"['baby⋅1',⋅'sorry⋅1']","[3,⋅4]"
simpsons⋅⋅⋅cat⋅lady,2,frog,table,eagle-tv-plus,1,3,"['baby⋅2',⋅'sorry⋅2']","[5,⋅6,⋅7]"
None,3,eagle,glass,lion-pc,c,4,"['baby⋅3',⋅'sorry⋅3']","[7,⋅8]"


In [105]:
df.cols.qcut("billingId", 5).table()

id 1 (int) nullable,firstName 2 (string) nullable,lastName 3 (string) nullable,billingId 4 (int) nullable,product 5 (string) nullable,price 6 (int) nullable,birth 7 (string) nullable,dummyCol 8 (string) nullable,billingId_QCUT 9 (double) nullable
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never,0.0
2,André,Ampère,423,piza,8,1950/07/08,gonna,1.0
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give,2.0
4,PAUL,dirac$,521,pizza,8,1954/07/10,you,2.0
5,Albert,Einstein,634,pizza,8,1990/07/11,up,3.0
6,Galileo,⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅GALiLEI,672,arepa,5,1930/08/12,never,3.0
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna,1.0
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let,2.0
9,Johannes,KEPLER,735,taco,3,1920/04/22,you,3.0
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down,4.0
